In [3]:
import pandas as pd

### combine my 3 CSV's into one CSV

In [3]:
df = pd.concat(map(pd.read_csv, ['Tournaments_data_3.csv', 'Tournaments_data_4.csv','Tournaments_data_5.csv']))

In [4]:
df.to_csv("combined_data.csv", encoding='utf-8')

In [5]:
df = df.set_index("Unique ID")

### Raw data to be brought in after combining CSV's

In [32]:
df = pd.read_csv("combined_data.csv", index_col="Unique ID")

In [2]:
df = pd.read_csv("testing_in_tableau.csv")

In [8]:
df = pd.read_csv("cleaned_poker_data.csv")

### Remove any null values
The null values we have are in the date column, as well as the game type column. Any date that was on the day that my script ran came back NaN, it doesn't seem to matter much to remove these as that day has past, and it will be just a few hundred lines. I am not sure why a few hundred came back without a game type, but they mostly seemed to be from the same locations. While it does suck to lose those locations, the game type is very key information, and I will be ommiting them unless I run the script again and can solve the issue of them coming back blank. This still leaves me with over 7k rows of information.

In [34]:
df = df.dropna()

### Convert date and time into readable DateTime

In [36]:
df["Date"] = df["Date"].str[1:5]
df["Date"] = df["Date"] + "/2023"

In [38]:
df['Start time'] = df['Start time'].str.replace("am", ":00 am")
df['Start time'] = df['Start time'].str.replace("pm", ":00 pm")

In [39]:
df["Date Time"] = df["Date"] + " " + df["Start time"]

In [41]:
df = df.drop(['Date', 'Start time'], axis=1)

### Create new column for the ratio of the buy in the goes to the prize pool
This step includes some slicing, as well as removing any rows with "Day 2" as a value in the "Total buy in" column. "Day 2" I decided was unnecessary, as that particular tournament will have been accounted for in its various day 1 entries. This left me with 5475 rows of data.

This step also includes turning the objects into numerical values

In [51]:
df = df[df["Total buy in"].str.contains("ay 2") == False]

In [46]:
df["Total buy in"] = df["Total buy in"].str[1:]
df["Prize pool"] = df["Prize pool"].str[1:]
df["rake"] = df["rake"].str[1:]

In [48]:
df["Total buy in"] = df["Total buy in"].str.replace(",", "")
df["Prize pool"] = df["Prize pool"].str.replace(",", "")


In [52]:
df["Total buy in"] = pd.to_numeric(df["Total buy in"])
df["Prize pool"] = pd.to_numeric(df["Prize pool"])

### Create "Prize pool ratio" column
This step is important, as it informs the poker player how much of their initial investment can potentially be returned, vs. how much of it goes to paying the venue and staff.

"Prize pool ratio" returned some NaN values as a freeroll tournament would try to find a value of 0/0. Therefore, I replaced NaN values with 1.1, denoting that their return is technically infinite because it requires 0 dollars to enter, but they can win dollars in return. It is important to note that freeroll tournaments are usually not worth the time when factored in with the opportunity cost of winning at a larger buy in tournament, if attempting to generate the highest hourly rate.

In [98]:
df["Prize pool ratio"] = df["Prize pool"] / df["Total buy in"]

In [72]:
df = df.fillna(1.1)

Additional info describes the information obtained in the collumn, as only some tournaments have a guarantee, but all tournaments have data here.

In [10]:
df = df.rename(columns={"Guarantee": "Additional info"})

### Create a postal code column to use in tableau later
- Step 1: Remove location
- Step 2: Export Distinct Venues to a CSV
- Step 3: Manually add Postal code looking it up on the internet
- Step 4: Join Venue_locations.csv and testing_in_tableau.csv

In [5]:
df = df.drop(["Unnamed: 0", "location"], axis=1)

In [ ]:
df["Venue"].value_counts().to_csv("Venue_locations.csv")

In [11]:
locations_df = pd.read_csv("Venue_locations.csv")

In [18]:
df = df.merge(right=locations_df, how='left', on='Venue')

In [12]:
df = df[['Date Time', 'Venue', 'Postal code', 'Game type', 'Total buy in', 'Prize pool', 'rake', 'Prize pool ratio', 'Additional info']]

In [14]:
df = df.rename(columns={"Postal code": "Postal Code", "Game type": "Game Type", 
"Prize pool": "Prize Pool", "rake": "Rake", "Prize pool ratio": "Prize Pool Ratio", "Additional info": "Additional Info"})

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5475 entries, 0 to 5474
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unique ID         5475 non-null   int64  
 1   Date Time         5475 non-null   object 
 2   Venue             5475 non-null   object 
 3   Postal Code       5475 non-null   int64  
 4   Game Type         5475 non-null   object 
 5   Total buy in      5475 non-null   int64  
 6   Prize Pool        5475 non-null   int64  
 7   Rake              5475 non-null   object 
 8   Prize Pool Ratio  5475 non-null   float64
 9   Additional Info   5475 non-null   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 427.9+ KB


In [17]:
df.to_csv("testing_in_tableau.csv", encoding='utf-8')

In [16]:
df.to_csv("cleaned_poker_data.csv", encoding='utf-8')